<a href="https://colab.research.google.com/github/Minsoo1036/Algorithm-Study/blob/main/rpy2_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --upgrade rpy2==3.4.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.4.5-cp39-cp39-linux_x86_64.whl size=297794 sha256=503ee3546242d3053465b41431af3a8ebd831d33811ef843ba09121ba00d7e4a
  Stored in directory: /root/.cache/pip/wheels/36/d1/63/882c1f63d21bcf817b7db960b9536a747d4258daeaace0edd4
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


In [1]:
import pandas as pd
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

from statsmodels.stats.anova import anova_lm 
import statsmodels.formula.api as smf

required_packages = ['base', 'forecast', 'CondIndTests','devtools'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

r = ro.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')
devtools = importr('devtools')

(as ‘lib’ is unspecified)




































































































































































	‘/tmp/RtmpgGxd7Z/downloaded_packages’



(as ‘lib’ is unspecified)







	‘/tmp/RtmpgGxd7Z/downloaded_packages’



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpgGxd7Z/remotesa7a722dd4aa/ericstrobl-RCIT-7a7fb2b/DESCRIPTION’ ... OK
* preparing ‘RCIT’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘RCIT_0.1.0.tar.gz’



(as ‘lib’ is unspecified)



In [6]:
required_packages = ['CausalImpact'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

CausalImpact = importr('CausalImpact')

(as ‘lib’ is unspecified)





























	‘/tmp/RtmpgGxd7Z/downloaded_packages’



In [8]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_process import ArmaProcess
#from causalimpact import CausalImpact


np.random.seed(12345)
ar = np.r_[1, 0.9]
ma = np.array([1])
arma_process = ArmaProcess(ar, ma)
X = 100 + arma_process.generate_sample(nsample=100)
y = 1.2 * X + np.random.normal(size=100)
y[70:] += 5

data = pd.DataFrame({'y': y, 'X': X}, columns=['y', 'X'])
pre_period = [0, 69]
post_period = [70, 99]


In [9]:
data

,y,X
0,118.188694,99.795292
1,120.233276,100.663180
2,118.627775,98.883699
3,119.609722,100.448941
4,121.391508,101.561734
...,...,...
95,131.838436,104.939544
96,118.786121,95.672520
97,127.362025,103.146201
98,123.600675,97.753389


In [11]:
with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_explorer = ro.conversion.py2rpy(data)

In [14]:
r_from_pd_explorer

y,X
...,...


In [22]:
time_points = r.seq(1,101,1)
r_from_pd_explorer = r.zoo(r_from_pd_explorer,time_points)

In [18]:
with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_explorer1 = ro.conversion.py2rpy(pre_period)

with localconverter(ro.default_converter + pandas2ri.converter):
    r_from_pd_explorer2 = ro.conversion.py2rpy(post_period)

In [32]:
r.unlist(r_from_pd_explorer1)

0,69


In [33]:
pre = r.unlist(r_from_pd_explorer1)

In [34]:
post = r.unlist(r_from_pd_explorer2)

In [36]:
fit = r.CausalImpact(r_from_pd_explorer,pre,post)

In [39]:
fit

series,[RTYPES.REALSXP]
summary,[RTYPES.VECSXP]
report,[RTYPES.STRSXP]
model,[RTYPES.VECSXP]


In [80]:
import numpy as np
tem2 = np.array(fit[0])

In [81]:
tem2.shape

(101, 14)

In [84]:
pd.DataFrame(tem2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,118.188694,118.188694,119.646839,117.440520,121.928148,118.188694,118.188694,118.188694,-1.458145,-3.739454,0.748174,0.000000,0.000000,0.000000
1,120.233276,238.421970,120.682802,118.645692,122.935833,238.421970,238.421970,238.421970,-0.449526,-2.702557,1.587584,0.000000,0.000000,0.000000
2,118.627775,357.049744,118.561580,116.321626,120.988713,357.049744,357.049744,357.049744,0.066195,-2.360938,2.306148,0.000000,0.000000,0.000000
3,119.609722,476.659467,120.431382,118.218294,122.608697,476.659467,476.659467,476.659467,-0.821660,-2.998975,1.391428,0.000000,0.000000,0.000000
4,121.391508,598.050975,121.760004,119.470380,124.152094,598.050975,598.050975,598.050975,-0.368495,-2.760586,1.921128,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,118.786121,11770.976293,114.885634,112.527934,117.292820,11638.323837,11621.241515,11653.440181,3.900487,1.493301,6.258188,132.652455,117.536111,149.734777
97,127.362025,11898.338317,123.799930,121.481860,126.159429,11762.123767,11744.454386,11778.267560,3.562095,1.202596,5.880165,136.214550,120.070758,153.883931
98,123.600675,12021.938992,117.368970,115.040082,119.606188,11879.492737,11861.458037,11895.002634,6.231705,3.994486,8.560593,142.446255,126.936358,160.480955
99,127.861827,12149.800819,122.642683,120.283927,124.792412,12002.135421,11983.921902,12018.441790,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
#pd.DataFrame(tem2).to_csv("causal.csv")